In [1]:
import numpy as np
import pandas as pd
from flood_tool import tool
from flood_tool import geo
tool=tool.Tool()

In [2]:
data=pd.read_csv('flood_tool/resources/stations.csv')

In [3]:
typical_day_df=pd.read_csv('flood_tool/resources/typical_day.csv')

In [4]:
wet_day_df=pd.read_csv('flood_tool/resources/wet_day.csv')

In [5]:
house_df=pd.read_csv('flood_tool/resources/households_per_sector.csv')

In [6]:
labeldata=pd.read_csv('flood_tool/resources/postcodes_sampled.csv')

In [7]:
l1=typical_day_df.stationReference.unique().tolist()#find all station_reference in typical.csv

In [8]:
l2=wet_day_df.stationReference.unique().tolist()#find all station_reference in wet.csv

In [9]:
z=list(set(l1)|set(l2))#merge all station_reference in both typical_day and wet_day 

In [10]:
d=data.stationReference.unique().tolist()#find all station_reference in stations.csv

In [11]:
difference = set(z).symmetric_difference(set(d))
list_difference = list(difference)#find the missing stationReference in stations.csv(soem of typical_day's 
#and wet_day's stationreference are not in stations.csv)

In [12]:
def findrow(i):#parameter:a station's stationReference
    url='https://environment.data.gov.uk/flood-monitoring/id/stations/'+i
    dfurl=pd.read_json(url)
    stationName=np.NaN
    if 'label' in dfurl.index:
        stationName=dfurl.loc['label']['items']
    latitude=np.NaN
    if 'lat' in dfurl.index:
         latitude=dfurl.loc['lat']['items']
    longitude=np.NaN
    if 'long' in dfurl.index:
        longitude=dfurl.loc['long']['items']
    maxOnRecord=np.NaN
    minOnRecord=np.NaN
    typicalRangeHigh=np.NaN
    typicalRangeLow=np.NaN
    if 'stageScale' in dfurl.index:
        if dfurl.loc['stageScale']['items'] != np.NaN:
            if 'typicalRangeHigh' in pd.read_json(url+'/stageScale').index :
                typicalRangeHigh=pd.read_json(url+'/stageScale').loc['typicalRangeHigh']['items']
                typicalRangeLow=pd.read_json(url+'/stageScale').loc['typicalRangeLow']['items']
                maxOnRecord=pd.read_json(url+'/stageScale').loc['maxOnRecord']['items']['value']
                minOnRecord=pd.read_json(url+'/stageScale').loc['minOnRecord']['items']['value']

    return pd.DataFrame(data={'stationReference':i,'stationName':[stationName],'latitude':[latitude],
                                    'longitude':[longitude],
                                    'maxOnRecord':[maxOnRecord],'minOnRecord':[minOnRecord],
                                    'typicalRangeHigh':[typicalRangeHigh],
                                   'typicalRangeLow':[typicalRangeLow]})

#return a row which has the same form as the stations.csv

In [13]:
findrow_df = pd.DataFrame(columns=['stationReference','stationName','latitude','longitude','maxOnRecord','minOnRecord',
                                   'typicalRangeHigh','typicalRangeLow'])#create an empty dataframe which column is the same as stations.csv
for i in list_difference:
    findrow_df=pd.concat([findrow_df, findrow(i)], axis=0)#merge all the missing stationrefrence's row into a dataframe

In [14]:
df=pd.concat([data.copy(),findrow_df],axis=0 ,ignore_index=True)#merge the stations.csv and findrow_df

In [15]:
df#df contains all the stations occurred in typical_day and wet_day

,stationReference,stationName,latitude,longitude,maxOnRecord,minOnRecord,typicalRangeHigh,typicalRangeLow
0,000008,Rainfall station,53.480556,-1.441674,NaN,NaN,NaN,NaN
1,000028,Rainfall station,53.500289,-1.673575,NaN,NaN,NaN,NaN
2,000075TP,Rainfall station,51.084022,-0.214597,NaN,NaN,NaN,NaN
3,000076TP,Rainfall station,51.701508,-0.747539,NaN,NaN,NaN,NaN
4,000180TP,Rainfall station,51.618838,0.173236,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2228,3401TH_AOD,Teddington Lock Head(AOD),51.429574,-0.318329,NaN,NaN,NaN,NaN
2229,L3370,Spurn Point,53.577909,0.107029,NaN,NaN,NaN,NaN
2230,E1997,Orton Sluice,52.560077,-0.281816,3.083,-0.012,2.26,0.911
2231,E72524,Hinkley Point,51.211000,-3.131000,NaN,NaN,NaN,NaN


In [16]:
def indicate_areas_at_risk(data):#parameter:any day's data
    stations_filled=df
    mean_typicalRangeHigh_randt=stations_filled.typicalRangeHigh.mean()#find the mean typicalRangeHigh of river,tidal 
    at_risk=[]
    try:
        for i in range(data.shape[0]):
            if data.loc[i].parameter=='rainfall':
                if float(data.loc[i].value)>=12.5:#we set any rainfall value reaches the violent level of rainfall is been risky
                    at_risk.append(data.loc[i].stationReference)
            else:
                if stations_filled[stations_filled.stationReference==data.loc[i].stationReference].typicalRangeHigh is np.NaN:
                    #if the station doesn't have typicalRangeHigh(which is quite normal) we use the mean value as a threshold
                    #above that threshold will be considered as risky
                    if float(data.loc[i].value)>=mean_typicalRangeHigh_randt:
                        at_risk.append(data.loc[i].stationReference)
                else:
                    if float(data.loc[i].value)>=(stations_filled[stations_filled.stationReference==data.loc[i].stationReference].typicalRangeHigh.values[0]):
                        #set any tidal or river value above that station's typicalRangeHigh as risky
                        at_risk.append(data.loc[i].stationReference)
    except ValueError:        
        return at_risk#return stationReference            

In [17]:
typical_day_risky_stationReference=indicate_areas_at_risk(typical_day_df)

In [18]:
wet_day_risky_stationReference=indicate_areas_at_risk(wet_day_df)

In [19]:
typical_day_risky_stationReference#no stations are detected as risky in the typical data

In [20]:
wet_day_risky_stationReference

['2660', '2830']

In [21]:
def impact_of_areas_at_risk(station_reference):#compute the impact of a given station reference
    lat=df[df.stationReference==station_reference].latitude.values
    long=df[df.stationReference==station_reference].longitude.values
    easting=geo.get_easting_northing_from_gps_lat_long(lat,long)[0]
    northing=geo.get_easting_northing_from_gps_lat_long(lat,long)[1]
    postcode=tool.get_postcode_from_OSGB36(easting, long).values[0]
    average_households=house_df[house_df['postcode sector']==postcode[:-2]].households/house_df[house_df['postcode sector']==postcode[:-2]]['number of postcode units']
    impact=(average_households*labeldata[labeldata.postcode==postcode].medianPrice.values).values
    return impact[0]

In [22]:
impact_typical_day=[]
try:
    for i in typical_day_risky_stationReference:
        impact_typical_day.append(impact_of_areas_at_risk(i))
except TypeError:
    None
impact_typical_day        

[]

In [23]:
impact_wet_day=[]
try:
    for i in wet_day_risky_stationReference:
        impact_wet_day.append(impact_of_areas_at_risk(i))
except TypeError:
    None
impact_wet_day

[2200154.1666666665, 4590502.994011977]

In [24]:
wet_lat=[]
wet_long=[]
for i in wet_day_risky_stationReference:
    wet_lat.append(df[df.stationReference==i].latitude.values[0])
    wet_long.append(df[df.stationReference==i].longitude.values[0])

In [25]:
risky_area_df=pd.DataFrame(data={'stationReference':wet_day_risky_stationReference,'lat':wet_lat,'long':wet_long,'impact':impact_wet_day})

In [26]:
risky_area_df#finally put the risky stationReference, lat, long and impact into a dataframe for visualizition

,stationReference,lat,long,impact
0,2660,51.907359,-2.061419,2.200154e+06
1,2830,52.735118,-2.960681,4.590503e+06


In [27]:
#risky_area_df.to_csv('risky_area.csv',index=False)